##### This Notebook use Doc AI and converts all PDF document to text so that down the line process can work

#### Author: Saurabh Mangal (saurabhmangal@google.com)
#### Editor / Reviewer: Wan Qi, Jing Le, Renzo Garcia
##### Date: 11th Mar
##### Description: This notebook contains part 1 of lab

 Copyright (c) [2024] [saurabhmangal@] -- 
 This notebook is licensed under the Commercial License.

### Install Dependencies & Restart Kernel

In [1]:
%pip install --quiet PyPDF2
%pip install --quiet pdfreader
!pip install --quiet google-cloud-discoveryengine
!pip install --upgrade --quiet google-cloud-storage
!pip3 install --upgrade --quiet google-cloud-documentai
!pip3 install --upgrade --quiet google-cloud-storage
!pip3 install --upgrade --quiet google-cloud-documentai-toolbox
%pip install --upgrade --quiet  google-cloud-documentai
%pip install --upgrade --quiet google-cloud-aiplatform -q
%pip install tqdm -q
%pip install langchain -q
!pip install --upgrade --quiet  langchain-google-genai
%pip install --upgrade --quiet  google-cloud-documentai
%pip install --upgrade --quiet  google-cloud-documentai-toolbox
%pip install --upgrade --quiet  langchain-core langchain-google-vertexai

#Restart kernel after installs so that your environment can access the new packages
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

time.sleep(10)

print("Installation done")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-python-client 1.8.0 requires google-api-core<2dev,>=1.13.0, but you have google-api-core 2.23.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-documentai-toolbox 0.14.0a0 requires google-cloud-documentai<3.0.0dev,>=2.29.2, but you have google-cloud-documentai 3.0.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated pack

In [1]:
## Run this cell and all below 

### Declare Variables

In [2]:
#no spaces or special characters allowed), ensure that it is unique
import socket
import re

UNIQUE_PREFIX = socket.gethostname()

UNIQUE_PREFIX = re.sub('[^A-Za-z0-9]+', '', UNIQUE_PREFIX)

PREFIX_NUM_ONLY=int(str(re.search(r'\d+', UNIQUE_PREFIX).group()))

REGION_ALLOCATE=PREFIX_NUM_ONLY%3

In [3]:
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="asia-southeast1"

FOLDER_NAME="."

### Create GCS Bucket & Import Documents

In [4]:
from google.cloud import storage

client = storage.Client()

GCS_BUCKET_LOCATION = LOCATION

GCS_BUCKET_NAME = f"{PROJECT_ID}-{UNIQUE_PREFIX}"
GCS_BUCKET_URI = f"gs://{GCS_BUCKET_NAME}"

bucket = storage.Bucket(client, GCS_BUCKET_NAME)

if bucket.exists()==False:
    # Create a Cloud Storage Bucket
    !gcloud storage buckets create $GCS_BUCKET_URI --location=$GCS_BUCKET_LOCATION

    # Upload the PDFs located in the books/ directory into the GCS bucket that you created
    !gsutil cp -r $FOLDER_NAME/books/* $GCS_BUCKET_URI/books

    # Verify that all Books 1 to 7 are uploaded to the GCS bucket (8 files in total, 2 for Part 1)
    !gsutil ls $GCS_BUCKET_URI
else:
    # Upload the PDFs located in the books/ directory into the GCS bucket that you created
    !gsutil cp -n $FOLDER_NAME/books/* $GCS_BUCKET_URI/books
    
    print(f"\n{GCS_BUCKET_NAME} already exists. Contents:\n")
    
    # Verify that all Books 1 to 7 are uploaded to the GCS bucket (8 files in total, 2 for Part 1)
    !gsutil ls $GCS_BUCKET_URI
    
def gcs_file(blob_name):
    return bucket.blob(blob_name)

Copying file://./books/PRUVantage-Wealth-Product-Brochure.pdf [Content-Type=application/pdf]...
Copying file://./books/ebrochure-en.pdf [Content-Type=application/pdf]...       
Copying file://./books/prufirst-promise_eBrochure_english.pdf [Content-Type=application/pdf]...
Copying file://./books/prulady-eBrochure-EN.pdf [Content-Type=application/pdf]...
- [4 files][ 13.1 MiB/ 13.1 MiB]  568.2 KiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./books/pruman-eBrochure-EN.pdf [Content-Type=application/pdf]...
Copying file://./books/prupersonal-accident-eBrochure-english.pdf [Content-Type=application/pdf]...
Copying file://./books/prushield-eBrochure-english.pdf [Content-Type=application/pdf]...
Copying file://./books/prushield_eBrochure

# Part 1: Document Processing

## Using Open Source Method to Process PDF Documents

In [6]:
from pdfreader import PDFDocument, SimplePDFViewer
from typing import Optional
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine

# Load the PDF document
pdf_file = gcs_file("books/ebrochure-en.pdf")

fd = pdf_file.open("rb")
doc = PDFDocument(fd)

from io import BytesIO
with pdf_file.open("rb") as f:
    stream = BytesIO(f.read())
doc2 = PDFDocument(stream)

page_one = next(doc.pages())

all_pages = [p for p in doc.pages()]
print(f"Number of pages: {len(all_pages)}")

Number of pages: 9


## Using Document AI to Process PDF Documents

In [7]:
# !mkdir ./books
# !mkdir ./matchingengine
# !mkdir ./matchingengine/embeddings

### Define helper functions for processors

In [8]:
# Function to create Document AI Processor
def create_processor(project_id, location, processor_display_name, processor_type):
    # You must set the api_endpoint if you use a location other than 'us'.
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the location
    # e.g.: projects/project_id/locations/location
    parent = client.common_location_path(project_id, location)

    # Create a processor
    processor = client.create_processor(
        parent=parent,
        processor=documentai.Processor(
            display_name=processor_display_name, type_=processor_type
        ),
    )
    
    processor_id = processor.name.split('/')[-1]

    # Print the processor information
    print(f"Processor Name: {processor.name}")
    print(f"Processor Display Name: {processor.display_name}")
    print(f"Processor ID: {processor_id}")
    print(f"Processor Type: {processor.type_}")
    
    
    return processor, processor_id

#Function to retrieve list of existing processors
def list_processors(project_id: str, location: str) -> None:
    processorList=[]
    
    # You must set the api_endpoint if you use a location other than 'us'.
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the location
    # e.g.: projects/project_id/locations/location
    parent = client.common_location_path(PROJECT_ID, location)

    # Make ListProcessors request
    processor_list = client.list_processors(parent=parent)

    # Print the processor information
    for processor in processor_list:
        # print(f"Processor Name: {processor.name}")
        # print(f"Processor Display Name: {processor.display_name}")
        # print(f"Processor Type: {processor.type_}")
        processorList.append(processor)
        
    return processorList

### Import Document AI libraries and set variables


In [9]:
from google.api_core.client_options import ClientOptions
from google.cloud import documentai

GCP_PROJECT = PROJECT_ID #'cloud-llm-preview1'
GCP_REGION='asia-southeast1'

# Variables for Document AI OCR Processor
PROCESSOR_DISPLAY_NAME = UNIQUE_PREFIX + '-ocr-processor' # Must be unique per project, e.g.: 'My Processor'
PROCESSOR_TYPE = 'OCR_PROCESSOR' # Use fetch_processor_types to get available processor types

### Create Document AI Document OCR Processor


In [10]:
LOCATION="us"

processorList=list_processors(PROJECT_ID,LOCATION)

def createUniqueProcessor():
    if len(processorList)==0:
        PROCESSOR, PROCESSOR_ID = create_processor(PROJECT_ID, LOCATION,PROCESSOR_DISPLAY_NAME, PROCESSOR_TYPE)
        return PROCESSOR, PROCESSOR_ID
    else:
        for processor in processorList:
            if PROCESSOR_DISPLAY_NAME==processor.display_name:
                PROCESSOR_ID=processor.name.split('/')[-1]
                PROCESSOR=processor
                return PROCESSOR, PROCESSOR_ID
            else:
                try:
                    PROCESSOR, PROCESSOR_ID = create_processor(PROJECT_ID, LOCATION,PROCESSOR_DISPLAY_NAME, PROCESSOR_TYPE)
                    return PROCESSOR, PROCESSOR_ID
                except:
                    continue
                

PROCESSOR, PROCESSOR_ID = createUniqueProcessor()


print(f"Processor {PROCESSOR_ID} already exists.")

Processor Name: projects/190852132224/locations/us/processors/5c6f0f728df4e3d8
Processor Display Name: aigpu1005fb7cc82-ocr-processor
Processor ID: 5c6f0f728df4e3d8
Processor Type: OCR_PROCESSOR
Processor 5c6f0f728df4e3d8 already exists.


### Processing a Single PDF Document using DocAI

In [11]:
from google.api_core.client_options import ClientOptions
from google.cloud import documentai


# PROJECT_ID = "YOUR_PROJECT_ID"
GCP_PROJECT= PROJECT_ID #'cloud-llm-preview1'
LOCATION = "us"  # Format is 'us' or 'eu'
PROCESSOR_ID = PROCESSOR_ID  # Create processor in Cloud Console
GCP_REGION="asia-southeast1"

# The local file in your current working directory
FILE_PATH = file_path = gcs_file("books/ebrochure-en.pdf")
# Refer to https://cloud.google.com/document-ai/docs/file-types
# for supported file types
MIME_TYPE = mime_type = "application/pdf"

# Instantiates a client
docai_client = documentai.DocumentProcessorServiceClient(
    client_options=ClientOptions(api_endpoint=f"{LOCATION}-documentai.googleapis.com")
)

# The full resource name of the processor, e.g.:
# projects/project-id/locations/location/processor/processor-id
# You must create new processors in the Cloud Console first
RESOURCE_NAME = docai_client.processor_path(PROJECT_ID, LOCATION, PROCESSOR_ID)

# Read the file into memory
with FILE_PATH.open("rb") as image:
    image_content = image.read()

# Load Binary Data into Document AI RawDocument Object
raw_document = documentai.RawDocument(content=image_content, mime_type=MIME_TYPE)

# Configure the process request
request = documentai.ProcessRequest(name=RESOURCE_NAME, raw_document=raw_document)

# Use the Document AI client to process the sample form
result = docai_client.process_document(request=request)

document_object = result.document
print("Document processing complete.")
print(f"Text: {document_object.text}")

page_text =document_object.text

Document processing complete.
Text: The plan is to
change the plan
PRUDENTIAL
PRUVantage Assure Series
Protect and grow your wealth
with a plan made for change
The plan is to change the plan
In this ever-changing world, disruptions to our lives and
plans can bring about unexpected changes and could put
a strain on our finances.
Hence, you must build a solid foundation to protect your
wealth so that you can confidently remain invested and
increase your wealth over time while conserving it for your
loved ones.
Stay agile with a plan that allows you to protect and grow
your wealth with ease in all market conditions. It's time to
change the way we invest, with a plan made for change -
PRUVantage Assure Series¹
Conserve
Preserve your legacy for the ones that
matters most to you
Grow
Maximise every opportunity to grow your
wealth with ease
Protect
De-risk your portfolio for greater security
in times of life's unexpected events
Made for change in an
ever-changing world
Make the most out of li

### Using batch mode for processing multiple documents

In [12]:
import re
from typing import Optional

from google.api_core.client_options import ClientOptions
from google.api_core.exceptions import InternalServerError
from google.api_core.exceptions import RetryError
from google.cloud import documentai  # type: ignore
from google.cloud import storage
import time

#Creating batch processing function
def batch_process_documents(
    project_id: str,
    location: str,
    processor_id: str,
    gcs_output_uri: str,
    processor_version_id: Optional[str] = None,
    gcs_input_uri: Optional[str] = None,
    input_mime_type: Optional[str] = None,
    gcs_input_prefix: Optional[str] = None,
    field_mask: "pages.pageNumber" = 221,
    timeout: int = 4000000000,
) -> None:
    # You must set the `api_endpoint` if you use a location other than "us".
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    if gcs_input_uri:
        # Specify specific GCS URIs to process individual documents
        gcs_document = documentai.GcsDocument(
            gcs_uri=gcs_input_uri, mime_type=input_mime_type
        )
        # Load GCS Input URI into a List of document files
        gcs_documents = documentai.GcsDocuments(documents=[gcs_document])
        input_config = documentai.BatchDocumentsInputConfig(gcs_documents=gcs_documents)
    else:
        # Specify a GCS URI Prefix to process an entire directory
        gcs_prefix = documentai.GcsPrefix(gcs_uri_prefix=gcs_input_prefix)
        input_config = documentai.BatchDocumentsInputConfig(gcs_prefix=gcs_prefix)

    # Cloud Storage URI for the Output Directory
    gcs_output_config = documentai.DocumentOutputConfig.GcsOutputConfig(
        gcs_uri=gcs_output_uri, field_mask=field_mask
    )

    # Where to write results
    output_config = documentai.DocumentOutputConfig(gcs_output_config=gcs_output_config)

    if processor_version_id:
        # The full resource name of the processor version, e.g.:
        # projects/{project_id}/locations/{location}/processors/{processor_id}/processorVersions/{processor_version_id}
        name = client.processor_version_path(
            project_id, location, processor_id, processor_version_id
        )
    else:
        # The full resource name of the processor, e.g.:
        # projects/{project_id}/locations/{location}/processors/{processor_id}
        name = client.processor_path(project_id, location, processor_id)

    request = documentai.BatchProcessRequest(
        name=name,
        input_documents=input_config,
        document_output_config=output_config,
    )

    # BatchProcess returns a Long Running Operation (LRO)
    operation = client.batch_process_documents(request)

    # Continually polls the operation until it is complete.
    # This could take some time for larger files
    # Format: projects/{project_id}/locations/{location}/operations/{operation_id}
    try:
        print(f"Waiting for operation {operation.operation.name} to complete...")
        operation.result(timeout=timeout)
    # Catch exception when operation doesn't finish before timeout
    except (RetryError, InternalServerError) as e:
        print(e.message)

    # Once the operation is complete,
    # get output document information from operation metadata
    metadata = documentai.BatchProcessMetadata(operation.metadata)

    if metadata.state != documentai.BatchProcessMetadata.State.SUCCEEDED:
        raise ValueError(f"Batch Process Failed: {metadata.state_message}")

    storage_client = storage.Client()

    print("Output files:")
    output_document = []
    # One process per Input Document
    for process in list(metadata.individual_process_statuses):
        # output_gcs_destination format: gs://BUCKET/PREFIX/OPERATION_NUMBER/INPUT_FILE_NUMBER/
        # The Cloud Storage API requires the bucket name and URI prefix separately
        matches = re.match(r"gs://(.*?)/(.*)", process.output_gcs_destination)
        if not matches:
            print(
                "Could not parse output GCS destination:",
                process.output_gcs_destination,
            )
            continue

        output_bucket, output_prefix = matches.groups()

        # Get List of Document Objects from the Output Bucket
        output_blobs = storage_client.list_blobs(output_bucket, prefix=output_prefix)
        
        # Document AI may output multiple JSON files per source file
        for blob in output_blobs:
            # Document AI should only output JSON files to GCS
            if blob.content_type != "application/json":
                print(
                    f"Skipping non-supported file: {blob.name} - Mimetype: {blob.content_type}"
                )
                continue

            # Download JSON File as bytes object and convert to Document Object
            print(f"Fetching {blob.name}")
            document = documentai.Document.from_json(
                blob.download_as_bytes(), ignore_unknown_fields=True
            )

            # For a full list of Document object attributes, please reference this page:
            # https://cloud.google.com/python/docs/reference/documentai/latest/google.cloud.documentai_v1.types.Document
            
            # Read the text recognition output from the processor 
            print("The document contains the following text:")
            print(document.text)
            output_document.append(document.text)
    return("".join(output_document))

def save_text_to_file(text, filename):
    pattern = r".*/([^/.]+)\.pdf"

    # Extract the filename
    match = re.search(pattern, filename)
    if match:
        filename = match.group(1)
        print(filename + " has been processed successfully.\n")  # Output ex: Book4_The_Goblet_of_Fire
    else:
        print("No match found")
    
    file = gcs_file("results/" + filename + ".txt")
    
    with file.open('w', encoding='utf-8') as f:
        f.write(text)    

### Setting Variables & Triggering the Batch Processing

In [46]:
!ls ./books/

PRUVantage-Wealth-Product-Brochure.pdf
ebrochure-en.pdf
prufirst-promise_eBrochure_english.pdf
prulady-eBrochure-EN.pdf
pruman-eBrochure-EN.pdf
prupersonal-accident-eBrochure-english.pdf
prushield-eBrochure-english.pdf
prushield_eBrochure_english.pdf


In [59]:
# PROJECT_ID = "YOUR_PROJECT_ID"
GCP_PROJECT= PROJECT_ID #'cloud-llm-preview1'
LOCATION = location = "us"  # Format is 'us' or 'eu'
project_id=PROJECT_ID
processor_id=PROCESSOR_ID

# The local file in your current working directory
FILE_PATH = f"{GCS_BUCKET_URI}/books/prushield_eBrochure_english.pdf"
# Refer to https://cloud.google.com/document-ai/docs/file-types
# for supported file types

# TODO(developer): Uncomment these variables before running the sample.
gcs_output_uri = f"{GCS_BUCKET_URI}/" # Must end with a trailing slash `/`. Format: gs://bucket/directory/subdirectory/
# processor_version_id = "" # Optional. Example: pretrained-ocr-v1.0-2020-09-23

# TODO(developer): You must specify either `gcs_input_uri` and `mime_type` or `gcs_input_prefix`
gcs_input_uri = f"{GCS_BUCKET_URI}/books/prushield-eBrochure-english.pdf" # Format: gs://bucket/directory/file.pdf
MIME_TYPE = input_mime_type = "application/pdf"

gcs_input_prefix = f"{GCS_BUCKET_URI}/matchingengine/" # Format: gs://bucket/directory/
field_mask = "text,entities,pages.pageNumber"  # Optional. The fields to return in the Document object.
timeout = 400000

book_list = [f"{GCS_BUCKET_URI}/books/prushield-eBrochure-english.pdf",
             # f"{GCS_BUCKET_URI}/books/prufirst-promise_eBrochure_english.pdf",
             # f"{GCS_BUCKET_URI}/books/PRUVantage-Wealth-Product-Brochure.pdf",
             f"{GCS_BUCKET_URI}/books/prulady-eBrochure-EN.pdf",
             f"{GCS_BUCKET_URI}/books/pruman-eBrochure-EN.pdf",
             # f"{GCS_BUCKET_URI}/books/ebrochure-en.pdf",
             f"{GCS_BUCKET_URI}/books/prupersonal-accident-eBrochure-english.pdf",]
try:
    for i in range(0,len(book_list)): 
        gcs_input_uri = book_list[i]
        print(gcs_input_uri + ":\n")

        page_text_batch = batch_process_documents(   project_id,
            location,
            processor_id,
            gcs_output_uri,
            None,
            gcs_input_uri,
            input_mime_type,
            gcs_input_prefix,
            field_mask,
            timeout)

        # Example usage: my_text = "This is the text from your OCR process."

        save_text_to_file(page_text_batch, gcs_input_uri)


        time.sleep(60)
except:
    pass

gs://prusandbx-nprd-uat-u9pahg-aigpu1005fb7cc82/books/prushield-eBrochure-english.pdf:

Waiting for operation projects/190852132224/locations/us/operations/7164588125558275386 to complete...
Output files:
Fetching 7164588125558275386/0/prushield-eBrochure-english-0.json
The document contains the following text:
Here, on your side
PRUDENTIAL
Supporting you as your trusted health partner,
so you can live your best life every step of
the way.
PRUShield. Better healthcare coverage, for better lives.
Medical bill
Why do you need additional medical coverage?
When the unexpected happens, you should focus on recovering instead of worrying about your medical bills.
Seeking treatment can cause a financial strain if you are not prepared for unexpected medical expenses,
especially with rising healthcare costs.
With Singapore being one of the fastest ageing populations in the world and more are living to 100¹, we are
committed to being your most trusted partner and protector by providing accessible

#### Here is the full documenation for DocAI: https://cloud.google.com/document-ai/docs/samples/documentai-batch-process-document?hl=en






## Part 2: Embeddings API

### Initialising Vertex AI & Setting Up Embeddings API

In [60]:
# init the vertexai package
import vertexai
LOCATION="us-central1"

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [61]:
# Load the text embeddings model
from vertexai.preview.language_models import TextEmbeddingModel 
embedding_model ="text-embedding-005"
model = TextEmbeddingModel.from_pretrained(embedding_model)

In [62]:
import time
import tqdm  # to show a progress bar
import os
# get embeddings for a list of texts
BATCH_SIZE = 5


def get_embeddings_wrapper(texts):
    embs = []
    for i in tqdm.tqdm(range(0, len(texts), BATCH_SIZE)):
        time.sleep(1)  # to avoid the quota error
        result = model.get_embeddings(texts[i : i + BATCH_SIZE])
        embs = embs + [e.values for e in result]
    return embs[0]

def text_embedding(text) -> list:
    """Text embedding with a Large Language Model."""
    model = TextEmbeddingModel.from_pretrained(embedding_model)
    embeddings = model.get_embeddings(text)
    for embedding in embeddings:
        vector = embedding.values
        print(f"Length of Embedding Vector: {len(vector)}")
    return vector

### Convert text into embeddings

In [63]:
trail_text = [page_text]

text_vectors = text_embedding(trail_text)

#Example of what the embeddings look like
for i in range(0,15):
    print(text_vectors[i])

Length of Embedding Vector: 768
-0.03582686185836792
0.02413112111389637
-0.05726239085197449
-0.039811134338378906
0.041710954159498215
0.025811096653342247
0.021746831014752388
-0.010525761172175407
-0.05459795147180557
0.011809822171926498
-0.04624815285205841
-0.06796979159116745
0.017386527732014656
-0.03221190720796585
0.01262790895998478


### Split text into lines (or sentences/words separated by symbols)

In [64]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=20,
    separators= ["/,", "##", ">", "-"],#'\n\n', '\n'],
    length_function=len,
    is_separator_regex=False,
)

splitted_texts = text_splitter.create_documents([page_text])
print("one \n",splitted_texts[0])
print("two \n",splitted_texts[1])

one 
 page_content='The plan is to
change the plan
PRUDENTIAL
PRUVantage Assure Series
Protect and grow your wealth
with a plan made for change
The plan is to change the plan
In this ever'
two 
 page_content='-changing world, disruptions to our lives and
plans can bring about unexpected changes and could put
a strain on our finances.
Hence, you must build a solid foundation to protect your
wealth so that you can confidently remain invested and
increase your wealth over time while conserving it for your
loved ones.
Stay agile with a plan that allows you to protect and grow
your wealth with ease in all market conditions. It's time to
change the way we invest, with a plan made for change '


### Display text that has been split in a DataFrame

In [65]:
splitted_texts_list = text_splitter.split_text(page_text)#[:2]

In [66]:
import pandas as pd

# Create a DataFrame from the splitted texts
df = pd.DataFrame({'splitted_texts': splitted_texts_list})

# Add a row number column
df['id'] = df.index + 1

# Print the DataFrame
print(df)

                                       splitted_texts  id
0   The plan is to\nchange the plan\nPRUDENTIAL\nP...   1
1   -changing world, disruptions to our lives and\...   2
2   -\nPRUVantage Assure Series¹\nConserve\nPreser...   3
3   -risk your portfolio for greater security\nin ...   4
4   -changing world\nMake the most out of life's u...   5
5   -linked plan\ndesigned to help you navigate a ...   6
6                                                 -in   7
7   -market Wealth Assure\nfeature that locks in c...   8
8                                                 -in   9
9   -market Wealth Assure feature\nwhich locks in ...  10
10  -ups or free partial withdrawals.\nEnjoy first...  11
11  -ups made and supplementary benefits. 3% yearl...  12
12  -owned Single Premium plan, only up to 3 times...  13
13  -smoker\nMax is a father of two and he is look...  14
14  -dividend paying fund in the Growth Account, w...  15
15  -ups or premiums paid for supplementary benefi...  16
16  -term need

### Display text that has been split in a List

In [67]:
list(df.splitted_texts)

['The plan is to\nchange the plan\nPRUDENTIAL\nPRUVantage Assure Series\nProtect and grow your wealth\nwith a plan made for change\nThe plan is to change the plan\nIn this ever',
 "-changing world, disruptions to our lives and\nplans can bring about unexpected changes and could put\na strain on our finances.\nHence, you must build a solid foundation to protect your\nwealth so that you can confidently remain invested and\nincrease your wealth over time while conserving it for your\nloved ones.\nStay agile with a plan that allows you to protect and grow\nyour wealth with ease in all market conditions. It's time to\nchange the way we invest, with a plan made for change ",
 '-\nPRUVantage Assure Series¹\nConserve\nPreserve your legacy for the ones that\nmatters most to you\nGrow\nMaximise every opportunity to grow your\nwealth with ease\nProtect\nDe',
 "-risk your portfolio for greater security\nin times of life's unexpected events\nMade for change in an\never",
 "-changing world\nMake the

# Part 3: Matching Engine

## Using Document AI Online Processing to process PDF document

### Declare DocAI Helper Functions

In [68]:
LOCATION = location = "us"
processor_version = 'rc'

def online_process(
    project_id: str,
    location: str,
    processor_id: str,
    file_path: str,
    mime_type: str,
) -> documentai.Document:
    """
    A function to process a document online using Google Document AI.
    """

    # Define an options dictionary, which includes the API's URL. This is used to connect to Google's Document AI service
    opts = {"api_endpoint": f"{location}-documentai.googleapis.com"}

    # Create a Document AI client, think of it as our bridge for communicating with Google's services
    documentai_client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # Generate the complete name of the processor
    # You need to first create a processor in the Google Cloud console
    resource_name = documentai_client.processor_path(project_id, location, processor_id)

    # Read in the document you want to analyze (like an image or PDF), and store it in the variable image_content
    with file_path.open("rb") as image:
        image_content = image.read()

        # Convert the read document into a format that Google Document AI can understand, i.e., a RawDocument object
        raw_document = documentai.RawDocument(
            content=image_content, mime_type=mime_type
        )
        # Create a request, which includes the name of the processor and the document we want to analyze
        request = documentai.ProcessRequest(
            name=resource_name, raw_document=raw_document
        )
        # Send our request and receive the analysis results
        result = documentai_client.process_document(request=request)
        
        print("Document processing complete.")
        # print(f"Text: {document_object.text}")
        
        # Return this analysis result
        return result.document

#Remove unwanted characters    
def trim_text(text: str): 
    """ Removes spaces and newline characters. """ 
    return text.strip().replace("\n", " ")

def process_document(
    project_id: str,
    location: str,
    processor_id: str,
    processor_version: str,
    file_path: str,
    mime_type: str,
    process_options: Optional[documentai.ProcessOptions] = None,
) -> documentai.Document:
    # You must set the `api_endpoint` if you use a location other than "us".
    client = documentai.DocumentProcessorServiceClient(
        client_options=ClientOptions(
            api_endpoint=f"{location}-documentai.googleapis.com"
        )
    )

    # The full resource name of the processor version, e.g.:
    # `projects/{project_id}/locations/{location}/processors/{processor_id}/processorVersions/{processor_version_id}`
    # You must create a processor before running this sample.
    name = client.processor_version_path(
        project_id, location, processor_id, processor_version
    )

    # Read the file into memory
    with open(file_path, "rb") as image:
        image_content = image.read()

    # Configure the process request
    request = documentai.ProcessRequest(
        name=name,
        raw_document=documentai.RawDocument(content=image_content, mime_type=mime_type),
        # Only supported for Document OCR processor
        process_options=process_options,
    )

    result = client.process_document(request=request)

    # For a full list of `Document` object attributes, reference this page:
    # https://cloud.google.com/document-ai/docs/reference/rest/v1/Document
    return result.document

### Extra DocAI Functions for reference (Will not be used in this lab)
#### Documentation: https://cloud.google.com/document-ai/docs/handle-response

In [69]:
# from typing import Optional, Sequence

# from google.api_core.client_options import ClientOptions
# from google.cloud import documentai

# def print_page_dimensions(dimension: documentai.Document.Page.Dimension) -> None:
#     print(f"    Width: {str(dimension.width)}")
#     print(f"    Height: {str(dimension.height)}")

# def print_detected_langauges(
#     detected_languages: Sequence[documentai.Document.Page.DetectedLanguage],
# ) -> None:
#     print("    Detected languages:")
#     for lang in detected_languages:
#         print(f"        {lang.language_code} ({lang.confidence:.1%} confidence)")


# def print_blocks(blocks: Sequence[documentai.Document.Page.Block], text: str) -> None:
#     print(f"    {len(blocks)} blocks detected:")
#     first_block_text = layout_to_text(blocks[0].layout, text)
#     print(f"        First text block: {repr(first_block_text)}")
#     last_block_text = layout_to_text(blocks[-1].layout, text)
#     print(f"        Last text block: {repr(last_block_text)}")


# def print_paragraphs(
#     paragraphs: Sequence[documentai.Document.Page.Paragraph], text: str
# ) -> None:
#     print(f"    {len(paragraphs)} paragraphs detected:")
#     first_paragraph_text = layout_to_text(paragraphs[0].layout, text)
#     print(f"        First paragraph text: {repr(first_paragraph_text)}")

#     last_paragraph_text = layout_to_text(paragraphs[-1].layout, text)
#     print(f"        Last paragraph text: {repr(last_paragraph_text)}")


# def print_lines(lines: Sequence[documentai.Document.Page.Line], text: str) -> None:
#     print(f"    {len(lines)} lines detected:")
#     first_line_text = layout_to_text(lines[0].layout, text)
#     print(f"        First line text: {repr(first_line_text)}")
#     last_line_text = layout_to_text(lines[-1].layout, text)
#     print(f"        Last line text: {repr(last_line_text)}")


# def print_tokens(tokens: Sequence[documentai.Document.Page.Token], text: str) -> None:
#     print(f"    {len(tokens)} tokens detected:")
#     first_token_text = layout_to_text(tokens[0].layout, text)
#     first_token_break_type = tokens[0].detected_break.type_.name
#     print(f"        First token text: {repr(first_token_text)}")
#     print(f"        First token break type: {repr(first_token_break_type)}")
#     if tokens[0].style_info:
#         print_style_info(tokens[0].style_info)

#     last_token_text = layout_to_text(tokens[-1].layout, text)
#     last_token_break_type = tokens[-1].detected_break.type_.name
#     print(f"        Last token text: {repr(last_token_text)}")
#     print(f"        Last token break type: {repr(last_token_break_type)}")
#     if tokens[-1].style_info:
#         print_style_info(tokens[-1].style_info)


# def print_symbols(
#     symbols: Sequence[documentai.Document.Page.Symbol], text: str
# ) -> None:
#     print(f"    {len(symbols)} symbols detected:")
#     first_symbol_text = layout_to_text(symbols[0].layout, text)
#     print(f"        First symbol text: {repr(first_symbol_text)}")
#     last_symbol_text = layout_to_text(symbols[-1].layout, text)
#     print(f"        Last symbol text: {repr(last_symbol_text)}")


# def print_image_quality_scores(
#     image_quality_scores: documentai.Document.Page.ImageQualityScores,
# ) -> None:
#     print(f"    Quality score: {image_quality_scores.quality_score:.1%}")
#     print("    Detected defects:")

#     for detected_defect in image_quality_scores.detected_defects:
#         print(f"        {detected_defect.type_}: {detected_defect.confidence:.1%}")


# def print_style_info(style_info: documentai.Document.Page.Token.StyleInfo) -> None:
#     """
#     Only supported in version `pretrained-ocr-v2.0-2023-06-02`
#     """
#     print(f"           Font Size: {style_info.font_size}pt")
#     print(f"           Font Type: {style_info.font_type}")
#     print(f"           Bold: {style_info.bold}")
#     print(f"           Italic: {style_info.italic}")
#     print(f"           Underlined: {style_info.underlined}")
#     print(f"           Handwritten: {style_info.handwritten}")
#     print(
#         f"           Text Color (RGBa): {style_info.text_color.red}, {style_info.text_color.green}, {style_info.text_color.blue}, {style_info.text_color.alpha}"
#     )


# def print_visual_elements(
#     visual_elements: Sequence[documentai.Document.Page.VisualElement], text: str
# ) -> None:
#     """
#     Only supported in version `pretrained-ocr-v2.0-2023-06-02`
#     """
#     checkboxes = [x for x in visual_elements if "checkbox" in x.type]
#     math_symbols = [x for x in visual_elements if x.type == "math_formula"]

#     if checkboxes:
#         print(f"    {len(checkboxes)} checkboxes detected:")
#         print(f"        First checkbox: {repr(checkboxes[0].type)}")
#         print(f"        Last checkbox: {repr(checkboxes[-1].type)}")

#     if math_symbols:
#         print(f"    {len(math_symbols)} math symbols detected:")
#         first_math_symbol_text = layout_to_text(math_symbols[0].layout, text)
#         print(f"        First math symbol: {repr(first_math_symbol_text)}")

# def layout_to_text(layout: documentai.Document.Page.Layout, text: str) -> str:
#     """
#     Document AI identifies text in different parts of the document by their
#     offsets in the entirety of the document"s text. This function converts
#     offsets to a string.
#     """
#     # If a text segment spans several lines, it will
#     # be stored in different text segments.
#     return "".join(
#         text[int(segment.start_index) : int(segment.end_index)]
#         for segment in layout.text_anchor.text_segments
#     )

### Trigger DocAI Online Document Processing

In [70]:
document = online_process(
    project_id=project_id,
    location=location,
    processor_id=processor_id,
    file_path=file_path,
    mime_type=mime_type,
)

names = []
name_confidence = []
values = []
value_confidence = []

Document processing complete.


## Using langchain for Doc AI on GCP

### Configure a DocAIParser 

In [71]:
# from langchain_community.document_loaders.blob_loaders import Blob
# from langchain_community.document_loaders.parsers import DocAIParser

# GCS_OUTPUT_PATH= GCS_BUCKET_URI 
# PROCESSOR_NAME = f'projects/{PROJECT_NUMBER}/locations/us/processors/{PROCESSOR_ID}'


# parser = DocAIParser(
#     location="us", processor_name=PROCESSOR_NAME, gcs_output_path=GCS_OUTPUT_PATH
# )

In [72]:


def combine_text_files(text_files):  
    combined_text = []
    for file in text_files:
        blob = gcs_file(file)
        text = blob.download_as_text()
        combined_text.append(text)
    return "\n\n".join(combined_text)

blob_names = [blob.name for blob in bucket.list_blobs(prefix = "results/")]

text = combine_text_files(blob_names)
page_contents = text_splitter.split_text(text)

### Display processed pages in a DataFrame

In [73]:
blob_names

['results/PRUVantage-Wealth-Product-Brochure.txt',
 'results/ebrochure-en.txt',
 'results/prufirst-promise_eBrochure_english.txt',
 'results/prulady-eBrochure-EN.txt',
 'results/pruman-eBrochure-EN.txt',
 'results/prupersonal-accident-eBrochure-english.txt',
 'results/prushield-eBrochure-english.txt']

In [74]:
import pandas as pd

# Create a DataFrame from the splitted texts
df = pd.DataFrame({'pagewise_texts': page_contents})

# Add a row number column
df['page_id'] = df.index + 1

# Print the DataFrame
print(df)

                                        pagewise_texts  page_id
0    Your next financial\nmasterpiece begins with\n...        1
1    -linked\nplan that empowers you to build\nweal...        2
2    -time 3% premium charge applies. I 4May be sub...        3
3    -Smoker.\nSara is a successful entrepreneur. S...        4
4    -paying fund¹\nDividend payout:\n4.5% p.a., re...        5
..                                                 ...      ...
394  -insurance is a percentage of the claimable am...      395
395  -pay after deductible.\n6 Foreigner with eligi...      396
396  -Term Visit Pass, Dependant's Pass and Work Pe...      397
397  -day free look period. Prudential will refund ...      398
398  -01 Marina One East Tower\nSingapore 018936\nT...      399

[399 rows x 2 columns]


In [75]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size=100
chunk_overlap=20

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators= ["/,", "##", ">", "We"],#'\n\n', '\n'],
    length_function=len,
    is_separator_regex=False,
)

def split_text_chunks(text, chunk_size):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

splitted_texts = []

for index, row in df.iterrows():
    splitted_texts += text_splitter.create_documents(row['pagewise_texts'])
    splitted_texts2 = text_splitter.split_text(row['pagewise_texts'])
    
    df['splitted_texts'] = df['pagewise_texts'].apply(lambda x: text_splitter.split_text(x))

### Display text that has been split in a DataFrame

In [76]:
df.head()

,pagewise_texts,page_id,splitted_texts
0,Your next financial\nmasterpiece begins with\n...,1,[Your next financial\nmasterpiece begins with\...
1,-linked\nplan that empowers you to build\nweal...,2,[-linked\nplan that empowers you to build\nwea...
2,-time 3% premium charge applies. I 4May be sub...,3,[-time 3% premium charge applies. I 4May be su...
3,-Smoker.\nSara is a successful entrepreneur. S...,4,[-Smoker.\nSara is a successful entrepreneur. ...
4,"-paying fund¹\nDividend payout:\n4.5% p.a., re...",5,"[-paying fund¹\nDividend payout:\n4.5% p.a., r..."


### Display text that has been split in a Table

In [77]:
df_exploded = df.explode('splitted_texts')
df_exploded.head()

,pagewise_texts,page_id,splitted_texts
0,Your next financial\nmasterpiece begins with\n...,1,Your next financial\nmasterpiece begins with\n...
0,Your next financial\nmasterpiece begins with\n...,1,"Wealth\nA resolute plan made to\ngrow, gain an..."
0,Your next financial\nmasterpiece begins with\n...,1,Wealth\nA uniquely crafted investment
1,-linked\nplan that empowers you to build\nweal...,2,-linked\nplan that empowers you to build\nweal...
1,-linked\nplan that empowers you to build\nweal...,2,Welcome Bonus and\nLoyalty Bonus of 0.8%² ever...


In [78]:
len(splitted_texts)

107128

### Split Text into Chunks, and add it to the Table as a column

In [79]:
df_exploded['splitted_texts_chunks'] = df_exploded['splitted_texts'].apply(lambda x: split_text_chunks(x,chunk_size))

In [80]:
df_exploded_2 = df_exploded.explode('splitted_texts_chunks')

In [81]:
df_exploded_2.head(10)

,pagewise_texts,page_id,splitted_texts,splitted_texts_chunks
0,Your next financial\nmasterpiece begins with\n...,1,Your next financial\nmasterpiece begins with\n...,Your next financial\nmasterpiece begins with\n...
0,Your next financial\nmasterpiece begins with\n...,1,Your next financial\nmasterpiece begins with\n...,g. Delivering.\nPRUVantage\n
0,Your next financial\nmasterpiece begins with\n...,1,"Wealth\nA resolute plan made to\ngrow, gain an...","Wealth\nA resolute plan made to\ngrow, gain an..."
0,Your next financial\nmasterpiece begins with\n...,1,"Wealth\nA resolute plan made to\ngrow, gain an...",h of opportunities for generations\nBehind eve...
0,Your next financial\nmasterpiece begins with\n...,1,"Wealth\nA resolute plan made to\ngrow, gain an...","election, calculated risks and the\nperseveran..."
0,Your next financial\nmasterpiece begins with\n...,1,"Wealth\nA resolute plan made to\ngrow, gain an...","e, dedication and determination\nneeded to suc..."
0,Your next financial\nmasterpiece begins with\n...,1,"Wealth\nA resolute plan made to\ngrow, gain an...",rth upholding with the\nutmost care and laying...
0,Your next financial\nmasterpiece begins with\n...,1,"Wealth\nA resolute plan made to\ngrow, gain an...","ur loved ones.\nAt Prudential, we believe each..."
0,Your next financial\nmasterpiece begins with\n...,1,"Wealth\nA resolute plan made to\ngrow, gain an...","andiwork over the years, with the\npotential t..."
0,Your next financial\nmasterpiece begins with\n...,1,"Wealth\nA resolute plan made to\ngrow, gain an...",s you to gain\ncertainty over the future and g...


### Index each row in the Table

In [82]:
df_exploded_2x = df_exploded_2.copy()

df_exploded_2x = df_exploded_2x.reindex()
df_exploded_2x = df_exploded_2x.reset_index()

df_exploded_2x['id'] = df_exploded_2x.index
df_exploded_2x['id'] = df_exploded_2x['id'].astype(str)

df_exploded_2 = df_exploded_2x.copy()

In [83]:
df_exploded_2.head()

,index,pagewise_texts,page_id,splitted_texts,splitted_texts_chunks,id
0,0,Your next financial\nmasterpiece begins with\n...,1,Your next financial\nmasterpiece begins with\n...,Your next financial\nmasterpiece begins with\n...,0
1,0,Your next financial\nmasterpiece begins with\n...,1,Your next financial\nmasterpiece begins with\n...,g. Delivering.\nPRUVantage\n,1
2,0,Your next financial\nmasterpiece begins with\n...,1,"Wealth\nA resolute plan made to\ngrow, gain an...","Wealth\nA resolute plan made to\ngrow, gain an...",2
3,0,Your next financial\nmasterpiece begins with\n...,1,"Wealth\nA resolute plan made to\ngrow, gain an...",h of opportunities for generations\nBehind eve...,3
4,0,Your next financial\nmasterpiece begins with\n...,1,"Wealth\nA resolute plan made to\ngrow, gain an...","election, calculated risks and the\nperseveran...",4


### Convert DataFrame (excluding Questions) into a CSV file
Split the dataframe into chunks and upload the file to cloud storage
(For the purposes of the workshop, we will be omitting this due to the time required to run this)

In [84]:
df_exploded_2.to_csv('df_exploded_2.csv', index=False)

### Import VertexAIEmbeddings and set the model

In [85]:
from langchain_google_vertexai import VertexAIEmbeddings

embeddings = VertexAIEmbeddings(model_name="text-embedding-005")

text = "This is a test document."

query_result = embeddings.embed_query(text)

### Import Question Bank from BigQuery public dataset

In [86]:
# load the BQ Table into a Pandas Dataframe
import pandas as pd
from google.cloud import bigquery

QUESTIONS_SIZE = df_exploded_2.shape[0] #1000

bq_client = bigquery.Client(project=PROJECT_ID)
QUERY_TEMPLATE = """
        SELECT distinct q.id, q.title
        FROM (SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions`
        where Score > 0 ORDER BY View_Count desc) AS q
        LIMIT {limit} ;
        """
query = QUERY_TEMPLATE.format(limit=QUESTIONS_SIZE)
query_job = bq_client.query(query)
rows = query_job.result()

In [87]:
len(df_exploded_2.splitted_texts_chunks)

1500

In [88]:
df_exploded_2.head()

,index,pagewise_texts,page_id,splitted_texts,splitted_texts_chunks,id
0,0,Your next financial\nmasterpiece begins with\n...,1,Your next financial\nmasterpiece begins with\n...,Your next financial\nmasterpiece begins with\n...,0
1,0,Your next financial\nmasterpiece begins with\n...,1,Your next financial\nmasterpiece begins with\n...,g. Delivering.\nPRUVantage\n,1
2,0,Your next financial\nmasterpiece begins with\n...,1,"Wealth\nA resolute plan made to\ngrow, gain an...","Wealth\nA resolute plan made to\ngrow, gain an...",2
3,0,Your next financial\nmasterpiece begins with\n...,1,"Wealth\nA resolute plan made to\ngrow, gain an...",h of opportunities for generations\nBehind eve...,3
4,0,Your next financial\nmasterpiece begins with\n...,1,"Wealth\nA resolute plan made to\ngrow, gain an...","election, calculated risks and the\nperseveran...",4


### Convert text from questions into Embeddings

In [89]:
import time
import tqdm  # to show a progress bar
from vertexai.language_models import TextEmbeddingModel

# get embeddings for a list of texts
BATCH_SIZE = 5


model_ai="text-embedding-005"

model = TextEmbeddingModel.from_pretrained(model_ai)

def get_embeddings_wrapper(texts):
    embs = []
    for i in tqdm.tqdm(range(0, len(texts), BATCH_SIZE)):
        time.sleep(1)  # to avoid the quota error
        result = model.get_embeddings(texts[i : i + BATCH_SIZE])
        embs = embs + [e.values for e in result]
    return embs
# The following code will get embedding for the question titles and add them as a new column embedding to the DataFrame. This will take a few minutes.

# get embeddings for the question titles and add them as "embedding" column
df = df_exploded_2.assign(embedding=get_embeddings_wrapper(list(df_exploded_2.splitted_texts_chunks)))
df.head()

100%|██████████| 300/300 [05:24<00:00,  1.08s/it]


,index,pagewise_texts,page_id,splitted_texts,splitted_texts_chunks,id,embedding
0,0,Your next financial\nmasterpiece begins with\n...,1,Your next financial\nmasterpiece begins with\n...,Your next financial\nmasterpiece begins with\n...,0,"[0.002942771650850773, 0.0003561877238098532, ..."
1,0,Your next financial\nmasterpiece begins with\n...,1,Your next financial\nmasterpiece begins with\n...,g. Delivering.\nPRUVantage\n,1,"[-0.04586053639650345, 0.006039049942046404, -..."
2,0,Your next financial\nmasterpiece begins with\n...,1,"Wealth\nA resolute plan made to\ngrow, gain an...","Wealth\nA resolute plan made to\ngrow, gain an...",2,"[-0.015320352278649807, -0.012343076057732105,..."
3,0,Your next financial\nmasterpiece begins with\n...,1,"Wealth\nA resolute plan made to\ngrow, gain an...",h of opportunities for generations\nBehind eve...,3,"[0.0013050033012405038, -0.005465648602694273,..."
4,0,Your next financial\nmasterpiece begins with\n...,1,"Wealth\nA resolute plan made to\ngrow, gain an...","election, calculated risks and the\nperseveran...",4,"[-0.007056122645735741, -0.012619102373719215,..."


### Convert Embeddings into a JSON File

In [90]:
# save id and embedding as a json file
jsonl_string = df[["id",'splitted_texts_chunks', "embedding"]].to_json(orient="records", lines=True)
with open(f"{FOLDER_NAME}/questions_test.json", "w") as f:
    f.write(jsonl_string)

# show the first few lines of the json file
! head -n 3 questions_test.json

{"id":"0","splitted_texts_chunks":"Your next financial\nmasterpiece begins with\none stroke\nof genius.\nPRUDENTIAL\nListening. Understandin","embedding":[0.0029427717,0.0003561877,-0.0473988205,-0.0226774495,0.0384290554,0.0369166546,0.0317643322,0.0058002193,-0.0131501611,-0.0032391488,-0.0766026378,-0.0470824651,0.0441718921,-0.0346134752,0.0334275849,0.0798263326,0.1143929064,-0.0162660535,-0.0165443979,-0.0086637326,0.0688555688,-0.0284351483,-0.0272004232,-0.0332174189,-0.0071514673,-0.0087814284,-0.0348908231,-0.0197900403,0.0075578829,0.0092782965,0.0239558835,-0.0173228737,-0.0219348241,-0.0109552881,0.0049809674,0.1050250381,0.0316631049,-0.0639405251,0.0123537118,-0.0140999341,0.0333486162,-0.0342945009,-0.0662196204,-0.0017379122,-0.0087388223,0.0425801203,-0.0139404703,0.0248527434,-0.0216100607,-0.0402237847,0.037030194,0.0177942719,-0.0412399732,0.027832374,0.0101572117,-0.0490783565,-0.0159034338,0.021168381,-0.0499353223,0.0070384489,-0.0737959817,-0.0715748444,-0.0479

### Upload the JSON File to matching engine

In [91]:
# generate an unique id for this session
from datetime import datetime

UID = UNIQUE_PREFIX

BUCKET_URI_ME=f"{GCS_BUCKET_URI}/matchingengine/embedding/"
LOCATION = LOCATION #'asia-southeast1'

from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [92]:
! gsutil cp questions_test.json {BUCKET_URI_ME}

Copying file://questions_test.json [Content-Type=application/json]...
\ [1 files][ 14.9 MiB/ 14.9 MiB]                                                
Operation completed over 1 objects/14.9 MiB.                                     


In [93]:
! gsutil ls {BUCKET_URI_ME}

gs://prusandbx-nprd-uat-u9pahg-aigpu1005fb7cc82/matchingengine/embedding/questions_test.json


## Creating Matching Engine Index

In [94]:
# create Index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=f"vs-feature-index-{UID}",
    contents_delta_uri=BUCKET_URI_ME,
    dimensions=768,
    approximate_neighbors_count=10,
    project = PROJECT_ID
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/190852132224/locations/asia-southeast1/indexes/8794061124719869952/operations/8364541855131500544
MatchingEngineIndex created. Resource name: projects/190852132224/locations/asia-southeast1/indexes/8794061124719869952
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/190852132224/locations/asia-southeast1/indexes/8794061124719869952')


### Create Index Endpoint and deploy the Index
To use the Index, you need to create an Index Endpoint. It works as a server instance accepting query requests for your Index.


In [95]:
# create IndexEndpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"vs-feature-index-endpoint-{UID}", public_endpoint_enabled=True
)

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/190852132224/locations/asia-southeast1/indexEndpoints/6519541002758258688/operations/2753056719427862528
MatchingEngineIndexEndpoint created. Resource name: projects/190852132224/locations/asia-southeast1/indexEndpoints/6519541002758258688
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/190852132224/locations/asia-southeast1/indexEndpoints/6519541002758258688')


In [96]:
DEPLOYED_INDEX_ID = f"vs_feature_deployed_{UID}"
# deploy the Index to the Index Endpoint
my_index_endpoint.deploy_index(index=my_index, deployed_index_id=DEPLOYED_INDEX_ID)

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/190852132224/locations/asia-southeast1/indexEndpoints/6519541002758258688
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/190852132224/locations/asia-southeast1/indexEndpoints/6519541002758258688/operations/2690006324644675584
MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/190852132224/locations/asia-southeast1/indexEndpoints/6519541002758258688


resource name: projects/190852132224/locations/asia-southeast1/indexEndpoints/6519541002758258688

In [97]:
# # delete Index Endpoint
# my_index_endpoint.undeploy_all()
# my_index_endpoint.delete(force=True)

# # delete Index
# my_index.delete()

#### Go to you vertex AI console and check the index is CREATED successfully 

### Querying a created index

In [98]:
import json

# build dicts for product names and embs
product_names = {}
product_embs = {}
product_text = {}
with open(f"{FOLDER_NAME}/questions_test.json") as f:
    for l in f.readlines():
        p = json.loads(l)
        id = p["id"]
        product_names[id] = p["id"]
        product_text[id] = p['splitted_texts_chunks']
        product_embs[id] = p["embedding"]

In [99]:
# get the embedding for ID 6523 "cloudveil women's excursion short"
# you can also try with other IDs such as 12711, 18090, 19536 and 11863
query_emb = product_embs["0"]

In [100]:
# run query
response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID, queries=[query_emb], num_neighbors=3
)

# show the results
for idx, neighbor in enumerate(response[0]):
    print(f"{neighbor.distance:.2f} {product_names[neighbor.id]} {product_text[neighbor.id]}")

1.00 0 Your next financial
masterpiece begins with
one stroke
of genius.
PRUDENTIAL
Listening. Understandin
0.69 2 Wealth
A resolute plan made to
grow, gain and give.
Discover the
Art of Investing
and create a wealt
0.67 7 ur loved ones.
At Prudential, we believe each portfolio
is a work of art. A true testament to your
h


### Run Query
Finally it's ready to use Vector Search. In the following code, it creates an embedding for a test question, and find similar question with the Vector Search.

In [101]:
import time
import tqdm  # to show a progress bar

# get embeddings for a list of texts
BATCH_SIZE = 5

# Load the text embeddings model
from vertexai.preview.language_models import TextEmbeddingModel

model = TextEmbeddingModel.from_pretrained("textembedding-gecko@003")

def get_embeddings_wrapper(texts):
    embs = []
    for i in tqdm.tqdm(range(0, len(texts), BATCH_SIZE)):
        time.sleep(1)  # to avoid the quota error
        result = model.get_embeddings(texts[i : i + BATCH_SIZE])
        embs = embs + [e.values for e in result]
    return embs

In [102]:
df = pd.read_csv('df_exploded_2.csv')

test_embeddings = get_embeddings_wrapper(["Who is the health insurance option for ladies?"])
# Test query
response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test_embeddings,
    num_neighbors=20,
)

# show the result
import numpy as np

for idx, neighbor in enumerate(response[0]):
    id = np.int64(neighbor.id)
    similar = df.query("id == @id", engine="python")
    print(f"{neighbor.distance:.4f} {similar.splitted_texts_chunks.values[0]}")


100%|██████████| 1/1 [00:02<00:00,  2.28s/it]


0.0873 -dividend paying fund in the Growth Account, with a premium term of 25 years.
Age
41
Invests a regul
0.0804 Wealth (USD) policy, which provides regular dividends to cover the costs of
running the studio, whil
0.0802 umber
of years, for as short as 8 years¹
Maximise your investments with 100% of
your premiums invest
0.0749  15 Any dividends distributed in the Growth Account will automatically be reinvested in the first 10
0.0735 iday charge and
defer administration charge for same period year
during premium term.
Enjoy flexibil
0.0729 
$284,674
Loyalty Bonus:
$1,517
Account Value:
$304,990
Starts receiving
21 0.5% Loyalty
Bonus annua
0.0701 Wealth Assure Values¹
are locked in at the highest daily Growth Account Value.
71
Peak Portfolio
Val
0.0699 hooses to reinvest the dividends she may receive
from the policy to accelerate wealth growth.
Annual
0.0675 g days².
Visit https://www.prudential.com.sg/ppc
0.0652 nds¹ received in Policy Year 11 to 30:
$160,174
Account Value:
$193,753
L

### Get an existing Index

In [103]:
from google.cloud import aiplatform

REGION = LOCATION # = "asia-southeast1"

aiplatform.init(project=PROJECT_ID, location=LOCATION)
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [104]:
my_index_name = my_index._gca_resource.name
my_index_display_name = my_index.display_name
my_index_id = my_index.name.split('/')[-1]

my_index_endpoint_name = my_index_endpoint._gca_resource.name
my_index_endpoint_display_name = my_index_endpoint.display_name
my_index_endpoint_id = my_index_endpoint.name.split('/')[-1]
my_index_endpoint_public_domain = my_index_endpoint.public_endpoint_domain_name

my_index = aiplatform.MatchingEngineIndex(my_index_name)

my_index_endpoint_id = my_index_endpoint_id

# my_index_endpoint_id = "[your-index-endpoint-id]"  # @param {type:"string"}
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(my_index_endpoint_id)

### Querying the index created earlier

In [105]:
from langchain_google_vertexai import VertexAIEmbeddings

embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@003")

In [106]:
# this is embedding vector (should be created by calling the embeddings models)

text = "insurance for men / boy"

test_embeddings = embeddings.embed_query(text)
print("preview embeddings",test_embeddings[0:2])

preview embeddings [-0.0111478166654706, -0.03209460899233818]


### Using Vector Search

In [107]:
# this setting is obtained from matching ending https://console.cloud.google.com/vertex-ai/locations/asia-southeast1/index-endpoints/3366088877738557440/deployed-indexes/vs_quickstart_deployed_02060053?project=jingle-project-414801

from google.cloud import aiplatform_v1

# Set variables for the current deployed index.
API_ENDPOINT=my_index_endpoint_public_domain
INDEX_ENDPOINT=my_index_endpoint_name

indexendpoint_id=UNIQUE_PREFIX

DEPLOYED_INDEX_ID="vs_feature_deployed_" + indexendpoint_id
neighbor_count = 3

print(API_ENDPOINT)
print(INDEX_ENDPOINT)
print(DEPLOYED_INDEX_ID)

2076259577.asia-southeast1-190852132224.vdb.vertexai.goog
projects/190852132224/locations/asia-southeast1/indexEndpoints/6519541002758258688
vs_feature_deployed_aigpu1005fb7cc82


In [108]:
# Configure Vector Search client
client_options = {
  "api_endpoint": API_ENDPOINT
}
vector_search_client = aiplatform_v1.MatchServiceClient(
  client_options=client_options,
)
# Build FindNeighborsRequest object
datapoint = aiplatform_v1.IndexDatapoint(
  feature_vector=test_embeddings
)

query = aiplatform_v1.FindNeighborsRequest.Query(
  datapoint=datapoint,
  # The number of nearest neighbors to be retrieved
  neighbor_count=neighbor_count
)

request = aiplatform_v1.FindNeighborsRequest(
  index_endpoint=INDEX_ENDPOINT,
  deployed_index_id=DEPLOYED_INDEX_ID,
  # Request can have multiple queries
  queries=[query],
  return_full_datapoint=False,
)

# Execute the request
response = vector_search_client.find_neighbors(request)

df_new = pd.DataFrame()
print('neighbor_count', neighbor_count)
for i in range(0,neighbor_count):
    x=response.nearest_neighbors[0]
    
    df_match = df.loc[df['id'] == int(x.neighbors[i].datapoint.datapoint_id) ]

    # Append the matching rows to the new DataFrame
    df_new = pd.concat([df_new, df_match])

# Print the new DataFrame
print(df_new)

neighbor_count 3
      index                                     pagewise_texts  page_id  \
65        5  -Smoker.\nReceives Welcome Bonus of\n70% of 1s...        6   
1499    398  -01 Marina One East Tower\nSingapore 018936\nT...      399   
317      40  -up. She is comfortable investing in the marke...       41   

                                         splitted_texts  \
65    Wealth (USD) policy, which provides regular di...   
1499  -01 Marina One East Tower\nSingapore 018936\nT...   
317   -up. She is comfortable investing in the marke...   

                                  splitted_texts_chunks    id  
65    Wealth (USD) policy, which provides regular di...    65  
1499                        www.prudential.com.sg\n26\n  1499  
317   Vantage Assure (SP), Brenda will be able to ca...   317  


In [109]:
def get_id_with_embedding_matching(test_embeddings) :
    
    datapoint = aiplatform_v1.IndexDatapoint(
      feature_vector=test_embeddings
    )
    query = aiplatform_v1.FindNeighborsRequest.Query(
      datapoint=datapoint,
      # The number of nearest neighbors to be retrieved
      neighbor_count=neighbor_count
    )
    request = aiplatform_v1.FindNeighborsRequest(
      index_endpoint=INDEX_ENDPOINT,
      deployed_index_id=DEPLOYED_INDEX_ID,
      # Request can have multiple queries
      queries=[query],
      return_full_datapoint=False,
    )

    # Execute the request
    response = vector_search_client.find_neighbors(request)
    
    df_new = pd.DataFrame()

    for i in range(0,neighbor_count):
        x=response.nearest_neighbors[0]

        df_match = df.loc[df['id'] == int(x.neighbors[i].datapoint.datapoint_id) ]

        # Append the matching rows to the new DataFrame
        df_new = pd.concat([df_new, df_match])
    
    print(df_new)
    i,j,k = df_new.index[0:3]
    print(i,j,k)
    
    pagewise_texts_v1 = df_new.loc[i, 'pagewise_texts']
    pagewise_texts_v2 = df_new.loc[j, 'pagewise_texts']
    pagewise_texts_v3 = df_new.loc[k, 'pagewise_texts']
    
    splitted_texts_v1 = df_new.loc[i, 'splitted_texts']
    splitted_texts_v2 = df_new.loc[j, 'pagewise_texts']
    splitted_texts_v3 = df_new.loc[k, 'pagewise_texts']
    
    splitted_texts_chunks_v1 = df_new.loc[i, 'splitted_texts_chunks']
    splitted_texts_chunks_v2 = df_new.loc[j, 'splitted_texts_chunks']
    splitted_texts_chunks_v3 = df_new.loc[k, 'splitted_texts_chunks']
    
    page_id_v1 = df_new.loc[i, 'page_id'] 
    page_id_v2 = df_new.loc[j, 'page_id'] 
    page_id_v3 = df_new.loc[k, 'page_id'] 
    
    return(pagewise_texts_v1,pagewise_texts_v2,pagewise_texts_v3,
           splitted_texts_v1,splitted_texts_v2,splitted_texts_v3,
           splitted_texts_chunks_v1,splitted_texts_chunks_v2,splitted_texts_chunks_v3,
        page_id_v1,page_id_v2,page_id_v3,i,j,k)

In [110]:
import pandas as pd
filename = f"{FOLDER_NAME}/pru_qa.csv"
df_qa = pd.read_csv(filename, sep ="|")

df_qa.head()

,Question,Answer
0,What is the name of the magical creature that ...,Thestral
1,What is the name of the school newspaper at Ho...,The Daily Prophet
2,What is the name of the magical map that shows...,Marauder's Map
3,Which Hogwarts house does Luna Lovegood belong...,Ravenclaw
4,What magical creature is known for guarding Gr...,Ukrainian Ironbelly (a dragon)


In [111]:
df_qa.columns

Index(['Question', 'Answer'], dtype='object')

### Export results into a CSV file

In [112]:
import csv

with open('pru_qa.csv', 'r') as input_file, open('pru_qa_output.csv', 'w', newline='') as output_file:

  # Create CSV reader and writer objects
  reader = csv.reader(input_file, delimiter='|')
  writer = csv.writer(output_file, delimiter='|')

  # Read and write the header row
  header = next(reader) + ['i','j','k','pagewise_texts_v1','pagewise_texts_v2','pagewise_texts_v3','splitted_texts_v1','splitted_texts_v2','splitted_texts_v3','splitted_texts_chunks_v1','splitted_texts_chunks_v2','splitted_texts_chunks_v3','page_id_v1','page_id_v2','page_id_v3']
  writer.writerow(header)

  # Loop through the remaining rows
  for i, row in enumerate(reader):
    question = row[0].split('|')[0]  # Use 'i' to access the correct element in the row
    question_emb = embeddings.embed_query( question )
    pagewise_texts_v1,pagewise_texts_v2,pagewise_texts_v3,splitted_texts_v1,splitted_texts_v2,splitted_texts_v3,splitted_texts_chunks_v1,splitted_texts_chunks_v2,splitted_texts_chunks_v3,page_id_v1,page_id_v2,page_id_v3,i,j,k = get_id_with_embedding_matching(question_emb) 
    
    # print( i , question)
    row_out = row + [i,j,k,pagewise_texts_v1,pagewise_texts_v2,pagewise_texts_v3,splitted_texts_v1,splitted_texts_v2,splitted_texts_v3,splitted_texts_chunks_v1,splitted_texts_chunks_v2,splitted_texts_chunks_v3,page_id_v1,page_id_v2,page_id_v3]
    
    # Write the row to the output file
    writer.writerow(row_out)

# Usage example:
! head -n 2 pru_qa_output.csv

     index                                     pagewise_texts  page_id  \
243     31  -dividend paying fund in the Growth Account, w...       32   
251     31  -dividend paying fund in the Growth Account, w...       32   
236     31  -dividend paying fund in the Growth Account, w...       32   

                                        splitted_texts  \
243  Wealth Assure Value\nSum Assured\nPeak Portfol...   
251  Wealth Assure Value; or (c) the account value ...   
236  -dividend paying fund in the Growth Account, w...   

                                 splitted_texts_chunks   id  
243  Wealth Assure Value\nSum Assured\nPeak Portfol...  243  
251  Wealth Assure Value; or (c) the account value ...  251  
236  -dividend paying fund in the Growth Account, w...  236  
243 251 236
     index                                     pagewise_texts  page_id  \
65       5  -Smoker.\nReceives Welcome Bonus of\n70% of 1s...        6   
601     79  -year premium of selected Prudential policies\... 

In [113]:
import pandas as pd
filename = f"{FOLDER_NAME}/pru_qa_output.csv"
df_qa = pd.read_csv(filename, sep ="|")

df_qa.head()

,Question,Answer,i,j,k,pagewise_texts_v1,pagewise_texts_v2,pagewise_texts_v3,splitted_texts_v1,splitted_texts_v2,splitted_texts_v3,splitted_texts_chunks_v1,splitted_texts_chunks_v2,splitted_texts_chunks_v3,page_id_v1,page_id_v2,page_id_v3
0,What is the name of the magical creature that ...,Thestral,243,251,236,"-dividend paying fund in the Growth Account, w...","-dividend paying fund in the Growth Account, w...","-dividend paying fund in the Growth Account, w...",Wealth Assure Value\nSum Assured\nPeak Portfol...,"-dividend paying fund in the Growth Account, w...","-dividend paying fund in the Growth Account, w...",Wealth Assure Value\nSum Assured\nPeak Portfol...,Wealth Assure Value; or (c) the account value ...,"-dividend paying fund in the Growth Account, w...",32,32,32
1,What is the name of the school newspaper at Ho...,The Daily Prophet,65,601,212,-Smoker.\nReceives Welcome Bonus of\n70% of 1s...,-year premium of selected Prudential policies\...,-ups made and supplementary benefits. 3% yearl...,"Wealth (USD) policy, which provides regular di...",-year premium of selected Prudential policies\...,-ups made and supplementary benefits. 3% yearl...,"Wealth (USD) policy, which provides regular di...",gery; Orchiectomy\nfor causes other than cance...,15 Any dividends distributed in the Growth Ac...,6,80,29
2,What is the name of the magical map that shows...,Marauder's Map,236,455,65,"-dividend paying fund in the Growth Account, w...","-term commitment, early termination of the pol...",-Smoker.\nReceives Welcome Bonus of\n70% of 1s...,"-dividend paying fund in the Growth Account, w...","-term commitment, early termination of the pol...",-Smoker.\nReceives Welcome Bonus of\n70% of 1s...,"-dividend paying fund in the Growth Account, w...","-term commitment, early termination of the pol...","Wealth (USD) policy, which provides regular di...",32,57,6
3,Which Hogwarts house does Luna Lovegood belong...,Ravenclaw,105,212,601,-year premium and varies based on chosen premi...,-ups made and supplementary benefits. 3% yearl...,-year premium of selected Prudential policies\...,-year premium and varies based on chosen premi...,-ups made and supplementary benefits. 3% yearl...,-year premium of selected Prudential policies\...,itial Investment Account after the administrat...,15 Any dividends distributed in the Growth Ac...,gery; Orchiectomy\nfor causes other than cance...,9,29,80
4,What magical creature is known for guarding Gr...,Ukrainian Ironbelly (a dragon),65,236,352,-Smoker.\nReceives Welcome Bonus of\n70% of 1s...,"-dividend paying fund in the Growth Account, w...","-55\n$2,400 p.a.\n$1,800 p.a.\nBased on latest...","Wealth (USD) policy, which provides regular di...","-dividend paying fund in the Growth Account, w...","-55\n$2,400 p.a.\n$1,800 p.a.\nBased on latest...","Wealth (USD) policy, which provides regular di...","-dividend paying fund in the Growth Account, w...",nvestment Account. A 3% premium charge will ap...,6,32,43


## Part 4: Trying Out Different AI Models

In [114]:
GCP_PROJECT=PROJECT_ID
LOCATION = REGION = 'asia-southeast1'

In [125]:
import os

import vertexai
from vertexai.preview.language_models import TextGenerationModel

vertexai.init(project=PROJECT_ID, location=LOCATION)


# @st.cache_resource
def get_model():
    generation_model = TextGenerationModel.from_pretrained("gemini-1.5-pro-002")
    return generation_model


def get_text_generation(prompt="", **parameters):
    generation_model = get_model()
    response = generation_model.predict(prompt=prompt, **parameters)

    return response.text

### Defining Functions for different models (Gemini, Unicorn, Bison32k)

In [126]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

def generate(input_prompt):
    model = GenerativeModel("gemini-ultra")
    responses = model.generate_content(
        input_prompt ,
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.2,
        "top_p": 1,
        "top_k": 32
    },
        safety_settings=[],
        stream=True,
    )
    
    all_response  = []
    
    for response in responses:
        # print(response.text, end="")
        all_response.append(response.text)
    
    # print (all_response)
    
    return(" ".join(all_response))
    

def generate_pro(input_prompt):
    model = GenerativeModel("gemini-1.5-pro-002")
    responses = model.generate_content(
    input_prompt,
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.2,
        "top_p": 1
    },stream=True,)
    
    all_response  = []
    
    for response in responses:
        all_response.append(response.text)
    
    # print (all_response)
    
    return(" ".join(all_response))


In [ ]:
!pip install --upgrade google-cloud-aiplatform

In [136]:
import vertexai
from vertexai.language_models import TextGenerationModel

vertexai.init(project=PROJECT_ID, location=LOCATION)
parameters = {
    "candidate_count": 1,
    "max_output_tokens": 1024,
    "temperature": 1,
    "top_k": 40
}

def generate_palm_unicorn_v1(input_prompt):
    
    model = TextGenerationModel.from_pretrained("gemini-1.5-pro-001")

    response = model.predict(
        input_prompt,
        **parameters
    )
    print(f"Response from Model: {response.text}")
    
    return(response.text)

def generate_palm_bison32k(input_prompt):
    
    model = TextGenerationModel.from_pretrained("gemini-pro")

    response = model.predict(
        input_prompt,
        **parameters
    )
    print(f"Response from Model: {response.text}")
    
    return(response.text)


### Read the Q&A file

#### This uses the file from Matching Engine which has questions and retrieved document results

In [137]:
import pandas as pd
import re
filename = f"{FOLDER_NAME}/pru_qa_output.csv"
df_qa = pd.read_csv(filename, sep ="|")

System_Prompts = """ You are an expert in reading insurance pdfs, but only provide evidences from the information provided and do not use any other information
so here are some search results : 
"""

Question_Prompts = """ -- Based on information above help to answer following user question
"""

df_qa['combine_prompt_RAG1'] = System_Prompts + ' ' +df_qa['pagewise_texts_v1'] + ' Please answers the Question : '+ df_qa['Question'] 
df_qa['combine_prompt_RAG2'] = System_Prompts + ' ' +df_qa['pagewise_texts_v2'] + ' Please answers the Question : '+ df_qa['Question'] 
df_qa['combine_prompt_RAG3'] = System_Prompts + ' ' +df_qa['pagewise_texts_v3'] + ' Please answers the Question : '+ df_qa['Question'] 


### Submit all the questions into different AI Models

In [139]:
for i in range(0, len(df_qa)):


    clean_text1 = re.sub(r'[^\w\s;]', '', df_qa.loc[i,'combine_prompt_RAG1'])
    clean_text2 = re.sub(r'[^\w\s;]', '', df_qa.loc[i,'combine_prompt_RAG2'])
    clean_text3 = re.sub(r'[^\w\s;]', '', df_qa.loc[i,'combine_prompt_RAG3'])

    if i<=1000:
        # df['Gemini_ultra_model_output'][i] = generate(df['combine_prompt'][i])
        print("iteration #", i, "test")
        if i==32 : 
            print("iteration #", i, "test", clean_text1, clean_text2, clean_text3)
    
    try:
        df_qa.loc[i, "Gemini_pro_model_output_v1"] = generate_pro(clean_text1)
        df_qa.loc[i, "Gemini_pro_model_output_v2"] = generate_pro(clean_text2)
        df_qa.loc[i, "Gemini_pro_model_output_v3"] = generate_pro(clean_text3)
    except :
        print("Prompt error at gemini ", i)
        df_qa.loc[i, "Gemini_pro_model_output_v1"] = "Prompt failed "
        df_qa.loc[i, "Gemini_pro_model_output_v2"] = "Prompt failed "
        df_qa.loc[i, "Gemini_pro_model_output_v3"] = "Prompt failed "
    

iteration # 0 test
iteration # 1 test
iteration # 2 test
iteration # 3 test
iteration # 4 test
iteration # 5 test
iteration # 6 test
iteration # 7 test
iteration # 8 test
iteration # 9 test
iteration # 10 test
iteration # 11 test
iteration # 12 test
iteration # 13 test
iteration # 14 test
iteration # 15 test
iteration # 16 test
iteration # 17 test
iteration # 18 test
iteration # 19 test
iteration # 20 test
iteration # 21 test
iteration # 22 test
iteration # 23 test
iteration # 24 test
iteration # 25 test
iteration # 26 test
iteration # 27 test
iteration # 28 test
iteration # 29 test
iteration # 30 test
iteration # 31 test
iteration # 32 test
iteration # 32 test  You are an expert in reading insurance pdfs but only provide evidences from the information provided and do not use any other information
so here are some search results  
 year premium and varies based on chosen premium term
40
50
60
70
Loyalty Bonus
Top up via Investment Booster
Continuing Investment Charge
Partial Withdrawal

In [140]:
df_qa.columns

Index(['Question', 'Answer', 'i', 'j', 'k', 'pagewise_texts_v1',
       'pagewise_texts_v2', 'pagewise_texts_v3', 'splitted_texts_v1',
       'splitted_texts_v2', 'splitted_texts_v3', 'splitted_texts_chunks_v1',
       'splitted_texts_chunks_v2', 'splitted_texts_chunks_v3', 'page_id_v1',
       'page_id_v2', 'page_id_v3', 'combine_prompt_RAG1',
       'combine_prompt_RAG2', 'combine_prompt_RAG3',
       'Gemini_pro_model_output_v1', 'Gemini_pro_model_output_v2',
       'Gemini_pro_model_output_v3'],
      dtype='object')

### Output all Questions & Answers from all AI Models into a CSV file

In [141]:
# Delete the 'col2' column
df_qa = df_qa.drop('combine_prompt_RAG1', axis=1)
df_qa = df_qa.drop('combine_prompt_RAG2', axis=1)
df_qa = df_qa.drop('combine_prompt_RAG3', axis=1)

output1 = f"{FOLDER_NAME}/results/pru_qa_model_out.csv"

df_qa.to_csv(output1)

In [142]:
df_qa.head(5)

,Question,Answer,i,j,k,pagewise_texts_v1,pagewise_texts_v2,pagewise_texts_v3,splitted_texts_v1,splitted_texts_v2,splitted_texts_v3,splitted_texts_chunks_v1,splitted_texts_chunks_v2,splitted_texts_chunks_v3,page_id_v1,page_id_v2,page_id_v3,Gemini_pro_model_output_v1,Gemini_pro_model_output_v2,Gemini_pro_model_output_v3
0,What is the name of the magical creature that ...,Thestral,243,251,236,"-dividend paying fund in the Growth Account, w...","-dividend paying fund in the Growth Account, w...","-dividend paying fund in the Growth Account, w...",Wealth Assure Value\nSum Assured\nPeak Portfol...,"-dividend paying fund in the Growth Account, w...","-dividend paying fund in the Growth Account, w...",Wealth Assure Value\nSum Assured\nPeak Portfol...,Wealth Assure Value; or (c) the account value ...,"-dividend paying fund in the Growth Account, w...",32,32,32,The provided text does not contain any inform...,The provided text does not contain any inform...,The provided text does not contain any inform...
1,What is the name of the school newspaper at Ho...,The Daily Prophet,65,601,212,-Smoker.\nReceives Welcome Bonus of\n70% of 1s...,-year premium of selected Prudential policies\...,-ups made and supplementary benefits. 3% yearl...,"Wealth (USD) policy, which provides regular di...",-year premium of selected Prudential policies\...,-ups made and supplementary benefits. 3% yearl...,"Wealth (USD) policy, which provides regular di...",gery; Orchiectomy\nfor causes other than cance...,15 Any dividends distributed in the Growth Ac...,6,80,29,The provided text does not contain any inform...,This document does not contain the answer to ...,This document does not contain information ab...
2,What is the name of the magical map that shows...,Marauder's Map,236,455,65,"-dividend paying fund in the Growth Account, w...","-term commitment, early termination of the pol...",-Smoker.\nReceives Welcome Bonus of\n70% of 1s...,"-dividend paying fund in the Growth Account, w...","-term commitment, early termination of the pol...",-Smoker.\nReceives Welcome Bonus of\n70% of 1s...,"-dividend paying fund in the Growth Account, w...","-term commitment, early termination of the pol...","Wealth (USD) policy, which provides regular di...",32,57,6,The provided text does not contain any inform...,The provided text does not contain any inform...,The provided text does not contain any inform...
3,Which Hogwarts house does Luna Lovegood belong...,Ravenclaw,105,212,601,-year premium and varies based on chosen premi...,-ups made and supplementary benefits. 3% yearl...,-year premium of selected Prudential policies\...,-year premium and varies based on chosen premi...,-ups made and supplementary benefits. 3% yearl...,-year premium of selected Prudential policies\...,itial Investment Account after the administrat...,15 Any dividends distributed in the Growth Ac...,gery; Orchiectomy\nfor causes other than cance...,9,29,80,The provided text does not contain any inform...,This PDF pertains to insurance plans and cont...,This PDF does not contain information about H...
4,What magical creature is known for guarding Gr...,Ukrainian Ironbelly (a dragon),65,236,352,-Smoker.\nReceives Welcome Bonus of\n70% of 1s...,"-dividend paying fund in the Growth Account, w...","-55\n$2,400 p.a.\n$1,800 p.a.\nBased on latest...","Wealth (USD) policy, which provides regular di...","-dividend paying fund in the Growth Account, w...","-55\n$2,400 p.a.\n$1,800 p.a.\nBased on latest...","Wealth (USD) policy, which provides regular di...","-dividend paying fund in the Growth Account, w...",nvestment Account. A 3% premium charge will ap...,6,32,43,The provided text does not contain any inform...,This document does not contain any informatio...,This PDF does not contain information about m...


### The response is not ideal due to the poor search setup. This can be finetuned further for better results.

# End of Lab